In [4]:
from PIL import Image
import pytesseract
from pytesseract import Output
import numpy as np
import re
import cv2
import spacy
import os 
import glob
import json
import pandas as pd
from pdf2image import convert_from_path

#cv2.imshow('image',image) # dipalying the image
#cv2.waitKey(0) 
#cv2.destroyAllWindows()

#directory = (r'Data/images/Structured/')

#for filename in os.listdir(directory):
#    if filename.endswith('.png'):
#         sample= Image.open(os.path.join("Data/images/Structured/", filename))

# img= cv2.imread(sample)  


####### Converting pdf2image #########
#pdfs =r"invoice3.pdf"
#pages= convert_from_path(pdfs,10)

#i=1
#for page in pages:
#    image_name="Page_"+ str(i) + ".jpg"
#    page.save(image_name,"JPEG")
#    i=i+1

   
def ocr_core(img):
    text = pytesseract.image_to_string(img)
    return text



#img = [cv2.imread(file) for file in glob.glob("Data/images/Structured/.png")]


img = cv2.imread('invoice.jpg') #reading the image

#PREPROCESSING THE IMAGE

def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#image1=get_grayscale(image)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,3)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 100, 230, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation = to adjust the font sizes
def dilate(image):
    kernel = np.ones((2,2),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion = = to adjust the font sizes
def erode(image):
    kernel = np.ones((2,2),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((1,1),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#resizeing the image
def resize(image):
    return cv2.resize(image,None,fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated


#canny edge detection == NOT RECOMMENDED RESULTING IN ACCURACY DROP
#def canny(image):
#    return cv2.Canny(image, 100, 200)

img= get_grayscale(img)
img= thresholding(img)
img= remove_noise(img)
img=erode(img)
img=dilate(img)
img= opening(img)
img=resize(img)

d = pytesseract.image_to_data(img, output_type=Output.DICT)
#print(d.keys())

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 2)


        


#cv2.imshow('img', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


custom_oem_psm_config =r'--oem 3 --psm 11'  # OEM CONTROLS TYPE OF Algo used and psm controls page segmentation
print(ocr_core(img))


text = pytesseract.image_to_string(img, lang='eng', config=custom_oem_psm_config)
nlp = spacy.load("en_core_web_sm")
sents = nlp(text)
count =1
data_list = []

#### Text Labelling on entities
for sent in sents:
    #print(sent)
    tokens = sent.text.split(" ")
    for i in range(len(tokens)):
        var = tokens[i]
        data = {}
        # print(var)
        if re.match(r'^[0-9]{2}[A-Z]{5}[0-9]{4}[A-Z]{1}[1-9A-Z]{1}Z[0-9A-Z]{1}$', str(var)):
            print("GST Number", str(var))
            data.update({"GST Number": str(var)})
        elif re.search(r'^[a-zA-Z]{5}[0-9]{4}[a-zA-Z]$', re.sub(r'\s+', '', str(var))):
                print("PAN Number", str(var))
                data.update({"PAN Number": str(var)})
            # Using Regular expression to match dates
        elif re.match(r'^(0[1-9]|[12][0-9]|3[01]).(0[1-9]|1[012]).(19|20)\d\d$', str(var)):
            print("Date of Invoice", str(var))
            data.update({"Date of Invoice": str(var)})
            #Simple number for amount
        elif re.match(r'\d{1,6}', str(var)):
            print("Amount", str(var))
            data.update({"Amount" : str(var)})

        

        count = count + 1
        if data != {}:
            data_list.append(data)
            
####  Exporting result to JSON FILE###            
#with open('data.json', 'w') as f:
#        json.dump(data_list, f)

print(data_list)

df =pd.DataFrame(data_list)
df.to_csv('output.csv')





1» O9AAACB1305Q1Z0
TAX INVOICE

BANSAL INDUSTRIES LIMITED (UNIT-III)
Trusted Wire house Since 1938 eee
Unit-III B-5 &) B-€] BITE-IT) Le (NDUSTRIAL AREA] M NAGAR] GHAZIABAD- 201007
Ten p2 fa bax TN 20-255873] Email
fice] F-3) Main Road)
CIN] GIN] u313000L198501c02" 737 RajAAACB1305Q

 

 

    
   

 

 

No) 1 GRIRR fol 0 §30003175664 29-01-2020
Data Of Supply =: Sees 1 Associated Road Carried Ltd]
FO) 1 (111029828 DT 27.01.2020 | Wehiclel No] : [INZ9ATS168
(Term) 1 : Glation 1 Madurat
Placa Bl 1 ;
SHIP
TVS CTD) SRICHAKRA LTD]
PERTIMAY) PATT] ROAD | PERUMAL PATT] ROAD
MELUR MELUR (TALUK
ADURAI-625124 [MADURAI-625122
PAN 0 AAACT® 557G 1 AAACTSS57G
STI 0 GSTIN : 33AAACT5S57G1ZE
€atatial Feriaved/aiepatched from thal premiseg ot the ) Worker M/s Aradhya Steel Pvt Ftd-} Sy} 154) 44 5y) 054) 2)
Road, Halavarthy, Anagodu, Anagodu, Davangere; Davangere; Karate a Ed Dy] nev 6ST] haziabad (U.P.)
— T
S.N, Description f FEN7SAG |lrosy Qty | Net Qty. unit Amount(7)
Oo a TYRE BEAD of BEAD IRI 72173

#Applying median blur
#image_blur=cv2.GaussianBlur(image2,(3,3),cv2.BORDER_DEFAULT)

#increaseing the size of our image
#image1=cv2.resize(image_blur, None, fx=2.0,fy=2.0,interpolation=cv2.INTER_CUBIC)

